# Are pathogenic mutations more likely to have modifications nearby?
In `studyBias_SwissProt.ipynb` we explored the relationship between annotations, PTMs, and mutations.  We found that there is a clear study bias for proteins that contain pathogenic mutations, having more GO terms, PTMs, and mutations.  Therefore, here we will consider only mutations coming from these heavily studied proteins as we explore the relationship between mutations and nearby PTMs.

In [1]:
# Setup the workspace, 
from proteomeScoutAPI import ProteomeScoutAPI
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
from scipy import stats 
import pickle
import random


[df_path, protID_path] = pickle.load(open("pathoProteins.p", "r"))
%matplotlib inline
proteomeScoutFile = '../../data/proteomescout_everything_20151118.tsv'

# read in ProteomeScout data
PTM_API = ProteomeScoutAPI(proteomeScoutFile)

# read in SwissProt mutations data 
SwissProtMuts = pickle.load(open('swissprot_mutations.p','rb'))

modWindow = 8 #change this to 
pValCutoff = 0.05

df_path.describe()



,GO,Mutations,PTMs,Sequence
count,2302.000000,2302.000000,2302.000000,2302.000000
mean,16.474805,15.945265,20.211121,753.383579
std,15.275281,27.289275,25.699069,725.794564
min,0.000000,1.000000,1.000000,51.000000
25%,7.000000,3.000000,6.000000,364.000000
50%,12.000000,7.000000,12.000000,529.500000
75%,21.000000,16.000000,25.000000,858.000000
max,159.000000,275.000000,316.000000,8797.000000


## Build mutations dataset
The following code moves through each protein we identified previously as having one or more pathogenic mutations, and for each mutations determines 
1. If it's a pathogenic mutation or not, and 
2. If it lies within a `modWindow` distance of a PTM, as well what that PTM is (assuming its one of phosphoserine, phosphothreonine, phosphotyrosine, ubiquitination, or 6-acetylylysine)

In [2]:
#### Create an object of all mutations that exist on proteins that have at least one pathogenic mutation.
# Also, add the number of nearby modifications of the major types, where nearby is ste on +/-7 amino acids
d = pd.DataFrame(columns=['ID', 'mod_pos', 'amino acid', 'pathoBit', 'Phosphoserine', 'Phosphothreonine', 'Phosphotyrosine',
'N6-acetyllysine', 'Ubiquitination', 'N-Glycosylation', 'O-Glycosylation'])

# for each protein
for ID in protID_path:
    
    # if we have mutation data
    if ID in SwissProtMuts:
        mutations = SwissProtMuts[ID]
 
        # for each mutation associated with protine $ID
        for mut in mutations:
            pos, from_res, to_res, patho_status = mut
        
            # get all PTMs which lie in a modWindow region (+/-) around
            # the mutation site
            mods = PTM_API.get_nearbyPTMs(ID, int(pos), modWindow)
        
            # if the mutation is associated with disease set tha pathoBit to 1,
            # else remains 0
            pathoBit = 0
            if patho_status == 'Disease':
                pathoBit = 1
    
            # create an initial protein record for the mutation, setting all 
            # modification counters to zero
            temp = pd.Series({'ID': ID, 'mod_pos':pos, 'amino acid':from_res, 'pathoBit':pathoBit, 
                            'Phosphoserine':0, 'Phosphothreonine':0, 'Phosphotyrosine':0, 'N6-acetyllysine':0, 
                            'Ubiquitination':0, 'N-Glycosylation':0, 'O-Glycosylation':0})
                    
            # for any modifications near the mutation (as defined by modWindow) increment
            # the appropriate modification counter
            for mod in mods:
                mod_pos, aa, mod_type = mod          
                try:
                    temp[mod_type] +=1
                except:
                    # we only care about 5 types of modification (others are to rare) so ignore
                    # other types
                    pass
                
            # finally append this record to the dataframe
            d = d.append(temp, ignore_index='True') 
    else:
        # no mutations, skip this protein!
        pass
        
d.sum()

ID                  P30613P30613P30613P30613P30613P30613P30613P306...
mod_pos                                                  1.961605e+07
amino acid          MVLERGDFIDGARRDGIRRNVTANNTQRRAGAVARRARRRRRVRNG...
pathoBit                                                        25849
Phosphoserine                                                    5445
Phosphothreonine                                                 2777
Phosphotyrosine                                                  2951
N6-acetyllysine                                                  1995
Ubiquitination                                                   2039
N-Glycosylation                                                  1476
O-Glycosylation                                                   214
dtype: object

### Checking for enrichment of mods near pathogenic mutations
Here is a description of the mutations and nearby modifications that make up the dataset we will consider, based on those annotations that come from proteins that have at least one pathogenic mutation.  We will use the Fisher Exact test to determine if there are enrichment differences. 

In [3]:
# Select only unique mutations based on protein and position.  Prioritize pathogenicity and binarize nearby mods 
dCollapse = pd.DataFrame(columns=d.columns)

for uniqueKey, row in d.groupby(['ID', 'mod_pos', 'amino acid']):
    temp = pd.DataFrame([[uniqueKey[0], uniqueKey[1], uniqueKey[2], int(row['pathoBit'].sum() > 0), 
                         int(row['Phosphoserine'].sum() > 0), int(row['Phosphothreonine'].sum() > 0), 
                        int(row['Phosphotyrosine'].sum() > 0), int(row['N6-acetyllysine'].sum() > 0), 
                       int(row['Ubiquitination'].sum() > 0), 
                         int(row['N-Glycosylation'].sum() > 0), 
                         int(row['O-Glycosylation'].sum() > 0), ]], columns=dCollapse.columns)
    dCollapse = dCollapse.append(temp)
dCollapse.describe()

,mod_pos,pathoBit,Phosphoserine,Phosphothreonine,Phosphotyrosine,N6-acetyllysine,Ubiquitination,N-Glycosylation,O-Glycosylation
count,32338.000000,32338.000000,32338.000000,32338.000000,32338.000000,32338.000000,32338.000000,32338.000000,32338.000000
mean,549.139712,0.684860,0.103748,0.061723,0.065465,0.039458,0.045519,0.039396,0.004082
std,697.511597,0.464579,0.304938,0.240656,0.247348,0.194685,0.208443,0.194539,0.063760
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,157.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,326.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,637.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8741.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
def FDR(pArr, alpha):
    """
    False Discovery Rate (FDR)
    Returns a Q value for significant values in pArr for a target FDR of alpha
    
    pArr  : Array of p-values
    alpha : Confidence threshold
    """
    
    # number of p-values
    m = len(pArr)
    
    # sort the p-values (smallest first)
    pValSort = sorted(pArr, key=lambda x: float(x))
        
    # assuming there are no rejected hypotheses at all
    pAdj = 0; 
    
    # cycle through each p-value
    for i in range (0, len(pValSort)):
        pi = pValSort[i]
        j = i+1;
        # print "Comparing %0.E to %0.2f"%(pi, (j*alpha/m))
        if(pi <= (j*alpha/m)):
            pAdj = pi
        else:
            # if the p-value is greater use the previously set
            # adjusted p-value
            return pAdj

In [5]:
def printPathoModStats(d):
    """
    Check for enrichment of pathogenic mutations near sites of modifications
    
    d :  binarized data frame defined above
    """

    # get number of mutation sites    
    N = len(d['pathoBit'])
    
    # get the number of mutation sites associated with disease    
    K = d['pathoBit'].sum()
    
    pvalueArr = []
    strSig = ''
    alpha = 0.05
    
    # for each of the modification types
    for mods in d.columns[-7:]:
        
        # number of mutation sites which have a modification 
        # of type $mods nearby
        n = len(d[d[mods]==1])

        # number of disease-related mutation which have a modification 
        # of type $mods nearby
        k = len(d[(d['pathoBit']==1) & (d[mods]==1)])
        
        # use the Fisher's exact test to take the following 
        # contigency table 
        # 
        #  
        #                        PATHOGENIC   NON-PATHOGENIC
        #  MUTATIONS NEAR SITE |   k              n-k
        #  MUTATIONS ANYWHERE  |   K              N-K
        #
        # And ask if the difference between the observations might occur by chance - the p
        # value reports on the likelyhood of finding pathogenic mutations near a site and non
        # pathogenic mutants away from a site occuring by chance vs. what we see - lower means
        # more sure it's a real effect
        oddsratio, pvalue = stats.fisher_exact([[k, n-k], [K, N-K]], alternative='greater')
        pvalueArr.append(pvalue)
        strSig = ''
        
        # for examples where the p-value is less than the defined alpha append a *
        if pvalue <= alpha:
            strSig = '*' 
        print "%2s %17s:\t %2.E \t(N=%d, n=%d, K=%d, k=%d)"%(strSig, mods, pvalue, N, n, K, k)
        
    # use the false discovery rate calculate the adjusted p-value
    p_adjust = FDR(pvalueArr, alpha)
    print "adjusted p-value is %0.3f"%(p_adjust)

        
    


In [6]:
printPathoModStats(dCollapse)

       Phosphoserine:	 1E+00 	(N=32338, n=3355, K=22147, k=2068)
    Phosphothreonine:	 1E+00 	(N=32338, n=1996, K=22147, k=1210)
 *   Phosphotyrosine:	 1E-05 	(N=32338, n=2117, K=22147, k=1543)
     N6-acetyllysine:	 2E-01 	(N=32338, n=1276, K=22147, k=890)
 *    Ubiquitination:	 4E-03 	(N=32338, n=1472, K=22147, k=1056)
     N-Glycosylation:	 7E-01 	(N=32338, n=1274, K=22147, k=863)
     O-Glycosylation:	 1E+00 	(N=32338, n=132, K=22147, k=61)
adjusted p-value is 0.004


# What is the distribution of amino acid types
Is there anything significant about the type of amino acids mutated and associated with pathogenicity?

In [7]:
# for each unique amino acid calculate whether there is over or under-representation (two-tailed test)

# Get the number of mutation sites and the number of pathogenic mutations (N and K respectively)
N = len(dCollapse['pathoBit'])
K = dCollapse['pathoBit'].sum()

runningSumCheck = 0
aaDict = {}

# cycle through each amino acid, collecting all the sites where
# that amino acid is mutated (row)
for uniqueKey, row in dCollapse.groupby(['amino acid']):
    
    # get current amino acids and number of mutation sites on that amino acid
    aa = uniqueKey[0]
    n = len(dCollapse[dCollapse['amino acid']==aa]) # aka len(row)
    
    # save the number of times that amino acid is observed on a mutation site
    aaDict[aa] = n
    
    # increment for book keeping
    runningSumCheck += n
    
    # get the number of occurences where a mutation to the $aa amino acid represents
    # a pathogenic mutation
    k = len(dCollapse[(dCollapse['amino acid']==aa) & (dCollapse['pathoBit']==1)])
    
    # use the Fisher's exact test to take the following 
    # contigency table 
    # 
    #  
    #                        PATHOGENIC   NON-PATHOGENIC
    #  MUTATIONS ON AA $aa |   k              n-k
    #  MUTATIONS ANYWHERE  |   K              N-K
    #
    # Smaller numbers mean we preferentially see pathogenic mutations on
    # some amino acid and less frequently see mutations on that amino
    # acid which are not pathogenic
    #    
    oddsratio, pvalue = stats.fisher_exact([[k, n-k], [K, N-K]])

    # Bonferroni correction, alpha at 0.05
    if pvalue <= pValCutoff/20: 
        print "%3s:\t %0.E \t(N=%d, n=%d, K=%d, k=%d)"%(aa, pvalue, N, n, K, k)
        
if runningSumCheck != N:
    print "Error in Parity: Sum is %d instead of %d"%(runningSumCheck, N)
    



  A:	 1E-16 	(N=32338, n=2328, K=22147, k=1398)
  C:	 5E-67 	(N=32338, n=1330, K=22147, k=1183)
  E:	 1E-04 	(N=32338, n=1542, K=22147, k=984)
  F:	 1E-05 	(N=32338, n=803, K=22147, k=607)
  G:	 7E-45 	(N=32338, n=3440, K=22147, k=2742)
  I:	 6E-08 	(N=32338, n=1255, K=22147, k=767)
  K:	 1E-09 	(N=32338, n=818, K=22147, k=476)
  L:	 5E-12 	(N=32338, n=2340, K=22147, k=1760)
  M:	 4E-04 	(N=32338, n=797, K=22147, k=498)
  N:	 4E-04 	(N=32338, n=1030, K=22147, k=651)
  P:	 2E-06 	(N=32338, n=1875, K=22147, k=1185)
  Q:	 2E-07 	(N=32338, n=816, K=22147, k=487)
  R:	 1E-04 	(N=32338, n=4784, K=22147, k=3408)
  S:	 7E-04 	(N=32338, n=1923, K=22147, k=1245)
  T:	 2E-18 	(N=32338, n=1622, K=22147, k=938)
  V:	 2E-31 	(N=32338, n=1871, K=22147, k=1033)
  W:	 1E-21 	(N=32338, n=524, K=22147, k=454)
  Y:	 5E-17 	(N=32338, n=912, K=22147, k=739)


## Examine random site-mutation enrichment
We checked to see if there was enrichment in nearby PTMs for any randomly selected mutation set based on selecting for the same distribution of amino acids. In other words, taking the same data set by randomly reshuffling which mutations were pathogenic and which were not, do we find a specific enrichment of pathogenic mutations near sites?

To do this, we create a new set based on selecting random permutations of the lsits that define particular amino acid types rebuild a fake patho set that is defined based on distribution of amino acids then check for enrichment of nearby mods


In [9]:
numRepeats = 10
print "Printing observed enrichments for random sets. Remember expected rate of seeing a p-value <=%0.2f is %1.2f times"%(pValCutoff, pValCutoff*5*numRepeats)
for i in range(1,numRepeats):
    
    # create an empty dataframe using the original column names
    dS = pd.DataFrame(columns=dCollapse.columns)
    
    # for each different amino acid
    for uniqueKey, row in dCollapse.groupby(['amino acid']):
        aa = uniqueKey[0]
        
        # this is just row (..)
        aaMuts = dCollapse[dCollapse['amino acid']==aa]

        # totally randomize the order of each of the mutation sites
        # associated with mutations of residue aa
        aaShuffled = aaMuts.iloc[np.random.permutation(aaDict[aa])]
        
        # have to explicity set index values so we can assign a subsection
        aaShuffled.index = range(0,len(aaShuffled))

        # Determine the number of pathogenic mutations associated with sites
        num = len(dCollapse[(dCollapse['amino acid']==aa) & (dCollapse['pathoBit']==1)])
        
        # set the first $num sites to be pathogenic and the rest to be not
        aaShuffled.loc[0:num, 'pathoBit'] = 1
        aaShuffled.loc[num:len(aaMuts), 'pathoBit'] = 0
        
        # sanity check!
        if num != aaShuffled['pathoBit'].sum():
            print "Problem for %s: wanted %d and retrieved %d"%(aa, num, aaShuffled['pathoBit'].sum())
            print "\t number of amino acids in set is %d and I have length of set %d"%(aaDict[aa], len(aaShuffled))
            
        # finally, append this set of data to the synthetic dataframe being created!
        dS = dS.append(aaShuffled)
    # now print enrichment
    print "Run %d"%(i)
    printPathoModStats(dS)

Printing observed enrichments for random sets. Remember expected rate of seeing a p-value <=0.05 is 2.50 times
Run 1
       Phosphoserine:	 9E-01 	(N=32338, n=3355, K=22147, k=2258)
    Phosphothreonine:	 1E+00 	(N=32338, n=1996, K=22147, k=1306)
     Phosphotyrosine:	 1E+00 	(N=32338, n=2117, K=22147, k=1408)
     N6-acetyllysine:	 1E+00 	(N=32338, n=1276, K=22147, k=844)
      Ubiquitination:	 9E-01 	(N=32338, n=1472, K=22147, k=982)
     N-Glycosylation:	 4E-01 	(N=32338, n=1274, K=22147, k=878)
     O-Glycosylation:	 1E+00 	(N=32338, n=132, K=22147, k=82)
adjusted p-value is 0.000
Run 2
       Phosphoserine:	 9E-01 	(N=32338, n=3355, K=22147, k=2261)
    Phosphothreonine:	 1E+00 	(N=32338, n=1996, K=22147, k=1321)
     Phosphotyrosine:	 8E-01 	(N=32338, n=2117, K=22147, k=1435)
     N6-acetyllysine:	 6E-01 	(N=32338, n=1276, K=22147, k=869)
      Ubiquitination:	 3E-01 	(N=32338, n=1472, K=22147, k=1018)
     N-Glycosylation:	 8E-01 	(N=32338, n=1274, K=22147, k=861)
     O-Glycosy